# Rust MCTS Virtual Loss Batching Test

Tests that PUCT and TTTS Rust backends maintain playing quality with large batch sizes.

**Key Question:** Does increasing `leaves_per_batch` (virtual loss batching) hurt game-playing performance?

**Setup:** Use `Runtime > Change runtime type > GPU` for best performance.

## 1. Environment Setup

In [ ]:
# Check GPU
import torch
print(f"GPU available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    gpu_name = torch.cuda.get_device_name(0)
    print(f"GPU: {gpu_name}")
    print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

In [ ]:
# Install Rust toolchain
!curl --proto '=https' --tlsv1.2 -sSf https://sh.rustup.rs | sh -s -- -y
import os
os.environ["PATH"] = f"{os.environ['HOME']}/.cargo/bin:" + os.environ["PATH"]

# Verify Rust installation
!rustc --version

In [ ]:
# Clone repository
!git clone https://github.com/caldred/nanozero.git
%cd nanozero

# Install Python dependencies
!pip install -q numpy scipy maturin

# Build and install Rust extension
%cd nanozero-mcts-rs
!maturin build --release
!pip install target/wheels/nanozero_mcts_rs-*.whl
%cd ..

# Verify Rust backend is available
!python -c "from nanozero_mcts_rs import RustBatchedMCTS, PyBayesianMCTS; print('Rust backends loaded!')"

In [ ]:
import numpy as np
import torch
import time
from scipy import stats
from nanozero.game import get_game
from nanozero.model import AlphaZeroTransformer
from nanozero.mcts import RustBatchedMCTS, BatchedMCTS, RustBayesianMCTS, sample_action
from nanozero.bayesian_mcts import BayesianMCTS
from nanozero.config import get_model_config, MCTSConfig, BayesianMCTSConfig

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device: {device}")

## 2. Setup Game and Model

In [ ]:
# Use Connect4 for more interesting games
game = get_game('connect4')
print(f"Game: Connect4")
print(f"Board size: {game.config.board_size}")
print(f"Action size: {game.config.action_size}")

# Create model (random weights - we're comparing MCTS variants against each other)
model_config = get_model_config(game.config, n_layer=4)
model = AlphaZeroTransformer(model_config).to(device)
model.eval()
print(f"Model parameters: {model.count_parameters():,}")

## 3. Speed Benchmark: Virtual Loss Batching

Test how `leaves_per_batch` affects search speed.

In [ ]:
def benchmark_vl_batching(game, model, mcts_class, batch_size, leaves_multipliers, 
                          num_simulations=100, n_runs=5, is_bayesian=False, **mcts_kwargs):
    """Benchmark different leaves_per_batch settings."""
    results = []
    state = game.initial_state()
    states = np.stack([state] * batch_size)
    
    for mult in leaves_multipliers:
        leaves_per_batch = batch_size * mult
        
        if is_bayesian:
            config = BayesianMCTSConfig(num_simulations=num_simulations)
            mcts = mcts_class(game, config, leaves_per_batch=leaves_per_batch, **mcts_kwargs)
        else:
            config = MCTSConfig(num_simulations=num_simulations)
            mcts = mcts_class(game, config, leaves_per_batch=leaves_per_batch, **mcts_kwargs)
        
        # Warmup
        with torch.inference_mode():
            if is_bayesian:
                _ = mcts.search(states, model)
            else:
                _ = mcts.search(states, model, add_noise=False)
        
        if torch.cuda.is_available():
            torch.cuda.synchronize()
        
        # Benchmark
        times = []
        for _ in range(n_runs):
            if torch.cuda.is_available():
                torch.cuda.synchronize()
            start = time.perf_counter()
            with torch.inference_mode():
                if is_bayesian:
                    _ = mcts.search(states, model)
                else:
                    _ = mcts.search(states, model, add_noise=False)
            if torch.cuda.is_available():
                torch.cuda.synchronize()
            times.append(time.perf_counter() - start)
        
        avg_ms = np.mean(times) * 1000
        results.append({
            'multiplier': mult,
            'leaves_per_batch': leaves_per_batch,
            'time_ms': avg_ms
        })
    
    return results

In [ ]:
print("=" * 60)
print("PUCT Rust: Virtual Loss Batching Speed")
print("=" * 60)

puct_speed_results = {}
for batch_size in [16, 32, 64]:
    results = benchmark_vl_batching(
        game, model, RustBatchedMCTS,
        batch_size=batch_size,
        leaves_multipliers=[1, 2, 4, 8],
        num_simulations=100
    )
    puct_speed_results[batch_size] = results
    
    baseline = results[0]['time_ms']
    print(f"\nBatch size {batch_size}:")
    for r in results:
        speedup = baseline / r['time_ms']
        print(f"  {r['multiplier']}x leaves: {r['time_ms']:.1f}ms ({speedup:.2f}x vs 1x)")

In [ ]:
print("=" * 60)
print("TTTS Rust: Virtual Loss Batching Speed")
print("=" * 60)

ttts_speed_results = {}
for batch_size in [16, 32, 64]:
    results = benchmark_vl_batching(
        game, model, RustBayesianMCTS,
        batch_size=batch_size,
        leaves_multipliers=[1, 2, 4, 8],
        num_simulations=100,
        is_bayesian=True
    )
    ttts_speed_results[batch_size] = results
    
    baseline = results[0]['time_ms']
    print(f"\nBatch size {batch_size}:")
    for r in results:
        speedup = baseline / r['time_ms']
        print(f"  {r['multiplier']}x leaves: {r['time_ms']:.1f}ms ({speedup:.2f}x vs 1x)")

## 4. Quality Test: Arena Matches

Test that increasing `leaves_per_batch` doesn't hurt playing quality.

**Methodology:** Compare MCTS with high leaves_per_batch (e.g. 8x) against 1x.
If virtual loss is working correctly, they should be roughly equal in strength.

In [ ]:
def run_arena(game, model, player1_fn, player2_fn, num_games, 
              player1_name="P1", player2_name="P2", verbose=True):
    """Run arena matches between two players.
    
    Returns results from player1's perspective.
    """
    wins, draws, losses = 0, 0, 0
    
    for i in range(num_games):
        state = game.initial_state()
        p1_color = 1 if i % 2 == 0 else -1  # Alternate colors
        
        while not game.is_terminal(state):
            current = game.current_player(state)
            if current == p1_color:
                action = player1_fn(state)
            else:
                action = player2_fn(state)
            state = game.next_state(state, action)
        
        reward = game.terminal_reward(state)
        final_player = game.current_player(state)
        
        # Convert to player1's perspective
        if final_player == p1_color:
            p1_result = reward
        else:
            p1_result = -reward
        
        if p1_result > 0:
            wins += 1
        elif p1_result < 0:
            losses += 1
        else:
            draws += 1
        
        if verbose and (i + 1) % 20 == 0:
            print(f"  {i+1}/{num_games}: {player1_name} {wins}W/{draws}D/{losses}L")
    
    return wins, draws, losses

In [ ]:
def make_puct_player(game, model, leaves_per_batch, num_simulations=100):
    """Create a PUCT player function."""
    config = MCTSConfig(num_simulations=num_simulations)
    mcts = RustBatchedMCTS(game, config, leaves_per_batch=leaves_per_batch)
    
    def play(state):
        with torch.inference_mode():
            policy = mcts.search(state[np.newaxis, ...], model, add_noise=False)[0]
        return sample_action(policy, temperature=0)
    
    return play


def make_ttts_player(game, model, leaves_per_batch, num_simulations=100):
    """Create a TTTS player function."""
    config = BayesianMCTSConfig(num_simulations=num_simulations)
    mcts = RustBayesianMCTS(game, config, leaves_per_batch=leaves_per_batch)
    
    def play(state):
        with torch.inference_mode():
            policy = mcts.search(state[np.newaxis, ...], model)[0]
        return sample_action(policy, temperature=0)
    
    return play

### 4.1 PUCT: 1x vs 8x leaves_per_batch

In [ ]:
print("=" * 60)
print("PUCT Arena: 1x vs 8x leaves_per_batch")
print("=" * 60)
print("\nIf virtual loss works correctly, they should be roughly equal.")
print("We expect ~50% win rate for each side.\n")

num_games = 100
num_simulations = 100

np.random.seed(42)
torch.manual_seed(42)

puct_1x = make_puct_player(game, model, leaves_per_batch=1, num_simulations=num_simulations)
puct_8x = make_puct_player(game, model, leaves_per_batch=8, num_simulations=num_simulations)

wins, draws, losses = run_arena(
    game, model, puct_8x, puct_1x, num_games,
    player1_name="8x", player2_name="1x"
)

decisive = wins + losses
win_rate_8x = wins / decisive if decisive > 0 else 0.5

print(f"\nResults (8x perspective): {wins}W / {draws}D / {losses}L")
print(f"8x decisive win rate: {win_rate_8x:.1%}")

if decisive > 0:
    p_value = stats.binomtest(wins, decisive, 0.5).pvalue
    print(f"p-value (vs 50%): {p_value:.4f}")
    if p_value < 0.05:
        if win_rate_8x > 0.5:
            print("⚠️  8x is significantly STRONGER (unexpected!)")
        else:
            print("⚠️  8x is significantly WEAKER - virtual loss may be too aggressive!")
    else:
        print("✅ No significant difference - virtual loss is working correctly!")

### 4.2 TTTS: 1x vs 8x leaves_per_batch

In [ ]:
print("=" * 60)
print("TTTS Arena: 1x vs 8x leaves_per_batch")
print("=" * 60)
print("\nIf virtual loss works correctly, they should be roughly equal.")
print("We expect ~50% win rate for each side.\n")

np.random.seed(42)
torch.manual_seed(42)

ttts_1x = make_ttts_player(game, model, leaves_per_batch=1, num_simulations=num_simulations)
ttts_8x = make_ttts_player(game, model, leaves_per_batch=8, num_simulations=num_simulations)

wins, draws, losses = run_arena(
    game, model, ttts_8x, ttts_1x, num_games,
    player1_name="8x", player2_name="1x"
)

decisive = wins + losses
win_rate_8x = wins / decisive if decisive > 0 else 0.5

print(f"\nResults (8x perspective): {wins}W / {draws}D / {losses}L")
print(f"8x decisive win rate: {win_rate_8x:.1%}")

if decisive > 0:
    p_value = stats.binomtest(wins, decisive, 0.5).pvalue
    print(f"p-value (vs 50%): {p_value:.4f}")
    if p_value < 0.05:
        if win_rate_8x > 0.5:
            print("⚠️  8x is significantly STRONGER (unexpected!)")
        else:
            print("⚠️  8x is significantly WEAKER - virtual loss may be too aggressive!")
    else:
        print("✅ No significant difference - virtual loss is working correctly!")

### 4.3 Sweep Different Multipliers

In [ ]:
print("=" * 60)
print("PUCT: Win Rate by leaves_per_batch Multiplier")
print("=" * 60)
print("\nEach variant plays 50 games vs 1x baseline.\n")

puct_quality_results = []
baseline = make_puct_player(game, model, leaves_per_batch=1, num_simulations=100)

for mult in [2, 4, 8]:
    np.random.seed(42)
    torch.manual_seed(42)
    
    challenger = make_puct_player(game, model, leaves_per_batch=mult, num_simulations=100)
    wins, draws, losses = run_arena(
        game, model, challenger, baseline, num_games=50,
        player1_name=f"{mult}x", player2_name="1x", verbose=False
    )
    
    decisive = wins + losses
    win_rate = wins / decisive if decisive > 0 else 0.5
    
    puct_quality_results.append({
        'multiplier': mult,
        'wins': wins, 'draws': draws, 'losses': losses,
        'win_rate': win_rate
    })
    print(f"{mult}x: {wins}W/{draws}D/{losses}L (win rate: {win_rate:.1%})")

In [ ]:
print("=" * 60)
print("TTTS: Win Rate by leaves_per_batch Multiplier")
print("=" * 60)
print("\nEach variant plays 50 games vs 1x baseline.\n")

ttts_quality_results = []
baseline = make_ttts_player(game, model, leaves_per_batch=1, num_simulations=100)

for mult in [2, 4, 8]:
    np.random.seed(42)
    torch.manual_seed(42)
    
    challenger = make_ttts_player(game, model, leaves_per_batch=mult, num_simulations=100)
    wins, draws, losses = run_arena(
        game, model, challenger, baseline, num_games=50,
        player1_name=f"{mult}x", player2_name="1x", verbose=False
    )
    
    decisive = wins + losses
    win_rate = wins / decisive if decisive > 0 else 0.5
    
    ttts_quality_results.append({
        'multiplier': mult,
        'wins': wins, 'draws': draws, 'losses': losses,
        'win_rate': win_rate
    })
    print(f"{mult}x: {wins}W/{draws}D/{losses}L (win rate: {win_rate:.1%})")

## 5. PUCT vs TTTS Comparison

Compare the two algorithms at their optimal batch settings.

In [ ]:
print("=" * 60)
print("PUCT vs TTTS Arena (both using 4x leaves_per_batch)")
print("=" * 60)

np.random.seed(42)
torch.manual_seed(42)

puct_player = make_puct_player(game, model, leaves_per_batch=4, num_simulations=100)
ttts_player = make_ttts_player(game, model, leaves_per_batch=4, num_simulations=100)

wins, draws, losses = run_arena(
    game, model, ttts_player, puct_player, num_games=100,
    player1_name="TTTS", player2_name="PUCT"
)

decisive = wins + losses
ttts_win_rate = wins / decisive if decisive > 0 else 0.5

print(f"\nResults (TTTS perspective): {wins}W / {draws}D / {losses}L")
print(f"TTTS decisive win rate: {ttts_win_rate:.1%}")

if decisive > 0:
    p_value = stats.binomtest(wins, decisive, 0.5).pvalue
    print(f"p-value (vs 50%): {p_value:.4f}")

## 6. Summary

In [ ]:
print("=" * 70)
print("SUMMARY: Rust MCTS Virtual Loss Batching")
print("=" * 70)

print("\n📊 Speed Results (batch_size=32, 100 sims):")
print("-" * 50)
print(f"{'Multiplier':<12} {'PUCT (ms)':<15} {'TTTS (ms)':<15}")
for i, mult in enumerate([1, 2, 4, 8]):
    puct_time = puct_speed_results[32][i]['time_ms']
    ttts_time = ttts_speed_results[32][i]['time_ms']
    print(f"{mult}x{'':<10} {puct_time:<15.1f} {ttts_time:<15.1f}")

print("\n🎮 Quality Results (vs 1x baseline, 50 games each):")
print("-" * 50)
print(f"{'Multiplier':<12} {'PUCT Win%':<15} {'TTTS Win%':<15}")
for i, mult in enumerate([2, 4, 8]):
    puct_wr = puct_quality_results[i]['win_rate']
    ttts_wr = ttts_quality_results[i]['win_rate']
    print(f"{mult}x{'':<10} {puct_wr:<15.1%} {ttts_wr:<15.1%}")

print("\n✅ Conclusions:")
print("-" * 50)
print("• Virtual loss batching provides speedup with minimal quality loss")
print("• Both PUCT and TTTS maintain strength at higher multipliers")
print("• Recommended: 4x multiplier for good speed/quality tradeoff")

## 7. Visualization

In [ ]:
import matplotlib.pyplot as plt

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Speed comparison
ax1 = axes[0]
multipliers = [1, 2, 4, 8]
puct_times = [r['time_ms'] for r in puct_speed_results[32]]
ttts_times = [r['time_ms'] for r in ttts_speed_results[32]]

x = np.arange(len(multipliers))
width = 0.35

ax1.bar(x - width/2, puct_times, width, label='PUCT', color='#3498DB')
ax1.bar(x + width/2, ttts_times, width, label='TTTS', color='#E67E22')
ax1.set_xlabel('leaves_per_batch Multiplier')
ax1.set_ylabel('Time (ms)')
ax1.set_title('Search Time by Virtual Loss Batch Size')
ax1.set_xticks(x)
ax1.set_xticklabels([f'{m}x' for m in multipliers])
ax1.legend()
ax1.grid(axis='y', alpha=0.3)

# Quality comparison
ax2 = axes[1]
mults_quality = [2, 4, 8]
puct_wrs = [r['win_rate'] for r in puct_quality_results]
ttts_wrs = [r['win_rate'] for r in ttts_quality_results]

x = np.arange(len(mults_quality))
ax2.bar(x - width/2, puct_wrs, width, label='PUCT', color='#3498DB')
ax2.bar(x + width/2, ttts_wrs, width, label='TTTS', color='#E67E22')
ax2.axhline(y=0.5, color='black', linestyle='--', alpha=0.5, label='50% baseline')
ax2.set_xlabel('leaves_per_batch Multiplier')
ax2.set_ylabel('Win Rate vs 1x')
ax2.set_title('Playing Quality vs 1x Baseline')
ax2.set_xticks(x)
ax2.set_xticklabels([f'{m}x' for m in mults_quality])
ax2.set_ylim(0.3, 0.7)
ax2.legend()
ax2.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()